In [1]:
#%%capture
%pip install accelerate peft bitsandbytes transformers trl
%pip install transformers
%pip install huggingface-hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 7.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

In [1]:
import os
import torch
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
from datasets import Dataset

from peft import PeftModel

In [2]:
from google.colab import userdata
from google.colab import drive
from sklearn.model_selection import train_test_split

In [14]:
import os

# Set the Hugging Face API token as an environment variable
os.environ["hf_token"] = "hf-token"

In [21]:
# Model from Hugging Face hub
base_model = "meta-llama/Llama-2-7b-hf"

# New instruction dataset
ads500B_dataset = "current_questions.txt"

# Fine-tuned model
new_model = "llama-2-7b-chat-ads500b"

In [18]:
from huggingface_hub import notebook_login

notebook_login()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
base_dir = '/content/'

In [23]:
def read_dataset(file_path):
    with open(file_path, 'r', encoding="cp1252") as file:
        lines = file.readlines()
    return lines

dataset = read_dataset(base_dir + ads500B_dataset)

In [24]:
def show_set(showset):
  for s in showset:
    print(s)

show_set(dataset[0:5])

### Human: What are the key software languages taught in ADS 500B course?, ### Assistant: The key software languages taught in ADS 500B course are Unix, SQL, R, and Python.

### Human: What is the focus of this course?, ### Assistant: This course is focused on practical aspects of exploratory data analysis, analytics, and basic machine learning techniques.

### Human: What types of questions are included in the quizzes of this course?, ### Assistant: The quizzes in this course consist of multiple choice and true and false questions.

### Human: What should students leverage to help them through the assignments and quizzes?, ### Assistant: Students should leverage the supplemental Unix material in Blackboard to help them through the assignments and quizzes.

### Human: What is recommended for students to install in preparation for Module 2?, ### Assistant: It is recommended for students to install the full graphical user interface version of Anaconda in preparation for Module 2.



In [25]:
for i, d in enumerate(dataset):
  d = d.replace(", ###", "\n###").strip()
  d = d[:-1]
  dataset[i] = d

show_set(dataset[0:5])

### Human: What are the key software languages taught in ADS 500B course?
### Assistant: The key software languages taught in ADS 500B course are Unix, SQL, R, and Python
### Human: What is the focus of this course?
### Assistant: This course is focused on practical aspects of exploratory data analysis, analytics, and basic machine learning techniques
### Human: What types of questions are included in the quizzes of this course?
### Assistant: The quizzes in this course consist of multiple choice and true and false questions
### Human: What should students leverage to help them through the assignments and quizzes?
### Assistant: Students should leverage the supplemental Unix material in Blackboard to help them through the assignments and quizzes
### Human: What is recommended for students to install in preparation for Module 2?
### Assistant: It is recommended for students to install the full graphical user interface version of Anaconda in preparation for Module 2


In [26]:
len(dataset)

1205

In [27]:
def split_dataset(data, test_size=0.1):
    train_set, test_set = train_test_split(data, test_size=test_size, random_state=42)
    return train_set, test_set

In [28]:
train_set, test_set = split_dataset(dataset)

In [29]:
df = pd.DataFrame(train_set, columns=['text'])

In [32]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(llm_int8_enable_fp32_cpu_offload=True)

In [33]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [34]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [35]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [36]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [37]:
dataset = Dataset.from_pandas(df)

In [38]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1084 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [39]:
trainer.train()

Step,Training Loss
25,1.780900
50,1.320500
75,1.264000
100,1.244000
125,1.222200
150,1.175400
175,1.124200
200,1.210600
225,1.155100
250,1.114600


TrainOutput(global_step=271, training_loss=1.254533176492501, metrics={'train_runtime': 701.448, 'train_samples_per_second': 1.545, 'train_steps_per_second': 0.386, 'total_flos': 2484280268980224.0, 'train_loss': 1.254533176492501, 'epoch': 1.0})

In [40]:
trainer.model.save_pretrained(base_dir+new_model)
trainer.tokenizer.save_pretrained(base_dir+new_model)

('/content/llama-2-7b-chat-ads500b/tokenizer_config.json',
 '/content/llama-2-7b-chat-ads500b/special_tokens_map.json',
 '/content/llama-2-7b-chat-ads500b/tokenizer.model',
 '/content/llama-2-7b-chat-ads500b/added_tokens.json',
 '/content/llama-2-7b-chat-ads500b/tokenizer.json')

In [44]:
# Reload model in FP16 and merge it with LoRA weights
load_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 250.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 1.06 MiB is free. Process 25486 has 14.71 GiB memory in use. Of the allocated memory 14.45 GiB is allocated by PyTorch, and 132.22 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [42]:
model = PeftModel.from_pretrained(load_model, base_dir+new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(base_model, token=hf_token, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

NameError: name 'load_model' is not defined

In [ ]:
model.save_pretrained(base_dir+new_model+"norm")
tokenizer.save_pretrained(base_dir+new_model+"norm")